In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)
iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

In [ ]:
!pip install chart-studio

In [1]:
import os
import math
import random
import numpy as np
from time import time
from pprint import pprint
from collections import Counter
from reconstruction.utils import binvox_rw
#from reconstruction.utils.plot import plot_vol
#import chart_studio.plotly as py

# Check datasets

In [2]:
path = 'C:/Users/Sebastian/Documents/ML-TF/ModelNet40/' #C:\Users\Sebastian\Documents\ML-TF
labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
files = [os.path.join(path,l,'train/',ll) for l in labels
         for ll in os.listdir(os.path.join(path, l, 'train/'))
         if ll[-4:] == '.off']

files = [f.replace("\\","/") for f in files]
pprint(Counter([f.split('ModelNet40/')[1].split('/')[0] for f in files]))

Counter({'chair': 889,
         'sofa': 680,
         'airplane': 626,
         'bookshelf': 572,
         'bed': 515,
         'vase': 475,
         'monitor': 465,
         'table': 392,
         'toilet': 344,
         'bottle': 335,
         'mantel': 284,
         'tv_stand': 267,
         'plant': 240,
         'piano': 231,
         'desk': 200,
         'dresser': 200,
         'night_stand': 200,
         'car': 197,
         'bench': 173,
         'glass_box': 171,
         'cone': 167,
         'tent': 163,
         'guitar': 155,
         'flower_pot': 149,
         'laptop': 149,
         'keyboard': 145,
         'curtain': 138,
         'sink': 128,
         'lamp': 124,
         'stairs': 124,
         'range_hood': 115,
         'door': 109,
         'bathtub': 106,
         'radio': 104,
         'xbox': 103,
         'stool': 90,
         'person': 88,
         'wardrobe': 87,
         'cup': 79,
         'bowl': 64})


In [ ]:
path = 'C:/Users/Sebastian/Documents/ML-TF/ModelNet10/' #C:/Users/Sebastian/Documents/ML-TF/ModelNet10/
labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
print(labels)
files = [os.path.join(path,l,'train/',ll) for l in labels
         for ll in os.listdir(os.path.join(path, l, 'train/'))
         if ll[-4:] == '.off']
files = [f.replace("\\","/") for f in files]
pprint(Counter([f.split('ModelNet10/')[1].split('/')[0] for f in files]))

# read / convert / plot

In [3]:
# generate file
file = random.choice(files)

def check_fix_file(file):
    with open(file) as f:
        l1 = f.readline()
        l2 = f.readlines()

    if l1 != 'OFF\n' and l1[:3] == 'OFF':
        out = 'OFF\n'
        out += l1.split('OFF')[1]
        out += ''.join(l2)
        with open(file, 'w') as f:
            f.write(out)

def voxels_from_file(file, voxsize):
    cmd = f'C:/Users/Sebastian/Documents/ML-TF/tools/binvox/binvox.exe -d {voxsize} -cb -e {file}' #tools/bonvox/bonvox.exe
    check_fix_file(file)
    out_file = file.split('.')[0] + '.binvox'
            
    if os.path.exists(out_file):
        os.remove(out_file)
    #print(cmd)
    t = os.system(cmd)
    #print(voxsize, file)
    #print(out_file, t)
    
    if t == 0:
        with open(out_file, 'rb') as f:
            d = binvox_rw.read_as_3d_array(f).data
        
        os.remove(out_file)
        return 1, d
    else:
        return 0, None

voxels = voxels_from_file(file, 32)
print('\nPlotting:', file)
print(voxels)
#plot_vol(voxels[1])


Plotting: C:/Users/Sebastian/Documents/ML-TF/ModelNet40/piano/train/piano_0216.off
(1, array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False

# Convert all ModelNet

In [4]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

def multithreading(func, args, workers):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        res = executor.map(func, args)
    return list(res)

get_label = lambda x: x.split('ModelNet')[1][3:].split('/')[0]

def get_voxels(files, voxsize):
    data = np.ndarray((0, *[voxsize]*3), dtype=bool)
    labels = []
    errors = []
    
    for i, file in enumerate(files):
        res = voxels_from_file(file, voxsize)
        if res[0] == 1:
            labels.append(get_label(file))
            data = np.vstack([data, res[1].reshape((1, *res[1].shape))])
        else:
            errors.append(file)

    return labels, data, errors

get_voxels_parallel = lambda x: get_voxels(*x)

def convert_all(path, voxsize):
    out_file = os.path.join(path, 'voxels.npy')
    labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    train_files = [os.path.join(path,l,'train',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'train'))
                   if ll[-4:] == '.off']
    test_files = [os.path.join(path,l,'test',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'test'))
                   if ll[-4:] == '.off']

    print(f'train: {len(train_files)}, test: {len(test_files)}')
    
    n_cpu = multiprocessing.cpu_count()
    output = {}
    
    for data_files, data_name in zip([train_files, test_files], ['train', 'test']):
        t0 = time()
        print(f'Launching {n_cpu} threads for {data_name} set...', end='')
        thread_size = math.ceil(len(data_files) / n_cpu)
        args = [(data_files[i*thread_size:(i+1)*thread_size], voxsize) for i in range(n_cpu)]
        res = multithreading(get_voxels_parallel, args, n_cpu)
        labels = []
        data = np.ndarray((0, *[voxsize]*3), dtype=bool)
        errors = []

        for l, d, e in res:
            labels += l
            data = np.vstack([data, d])
            errors += e
            
        output[data_name] = {'labels': labels, 'data': data, 'errors': errors}
        
        print('(%.2fs)' % (time() - t0))
    
    np.save(out_file, output)    
    print('\nSaved on: %s (%.2fM)' % (out_file, os.path.getsize(out_file) / 1024**2))

In [5]:
convert_all('C:/Users/Sebastian/Documents/ML-TF/ModelNet40/', voxsize=32) # C:/Users/Proyecto/Documents/ML/repo/ModelNet40/

train: 9843, test: 2468
Launching 8 threads for train set...(795.65s)
Launching 8 threads for test set...(146.13s)

Saved on: C:/Users/Sebastian/Documents/ML-TF/ModelNet40/voxels.npy (385.16M)


In [6]:
convert_all('C:/Users/Sebastian/Documents/ML-TF/ModelNet10/', voxsize=32)

train: 3991, test: 908
Launching 8 threads for train set...(177.17s)
Launching 8 threads for test set...(34.11s)

Saved on: C:/Users/Sebastian/Documents/ML-TF/ModelNet10/voxels.npy (153.26M)


In [8]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [12]:
modelnet10 = np.load('C:/Users/Sebastian/Documents/ML-TF/ModelNet10/voxels.npy').item()

#print(len(modelnet10['train']['labels']))
idx = random.choice(range(len(modelnet10['train']['labels'])))
#print(idx)
print(modelnet10['train']['labels'][idx])
#plot_vol(modelnet10['train']['data'][idx])

night_stand\train\night_stand_0136.off


In [11]:
modelnet40 = np.load('C:/Users/Sebastian/Documents/ML-TF/ModelNet40/voxels.npy').item()

idx = random.choice(range(len(modelnet40['train']['labels'])))
print(modelnet40['train']['labels'][idx])
#plot_vol(modelnet40['train']['data'][idx])

table\train\table_0008.off
